# Discover & Process Sentinel-1 InSAR pairs

- The provider used for this notebook is `cop_dataspace`
- A free account needs to be created at https://dataspace.copernicus.eu/
- The credentials need to be configured as explained here https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import explore_products
from eo_tools.util import show_insar_coh, show_insar_phi, show_sar_int, show_cog
from eo_tools.S1 import process_InSAR
import geopandas as gpd
import folium
import folium.plugins
import logging
import os

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Select an area of interest

```
{"type":"Polygon","coordinates":[[[2.210312,50.68232],[2.210312,50.766648],[2.368584,50.766648],[2.368584,50.68232],[2.210312,50.68232]]]}
```

In [ ]:
from shapely.geometry import shape

# France - 06-01-2024
# shp = shape({"type":"Polygon","coordinates":[[[2.210312,50.68232],[2.210312,50.766648],[2.368584,50.766648],[2.368584,50.68232],[2.210312,50.68232]]]})

# Netherlands - 19-07-2021
shp = shape({"type":"Polygon","coordinates":[[[5.924721,51.141125],[5.924721,51.203764],[6.037846,51.203764],[6.037846,51.141125],[5.924721,51.141125]]]})

## Search

In [ ]:
# load a geometry
search_criteria = {
    # "productType": "S1_SAR",
    "productType": "S1_SAR_SLC",
    "start": "2021-06-20",
    "end": "2021-07-25",
    "geom": shp,
}
results, _ = dag.search(**search_criteria)

In [ ]:
results[12]

In [ ]:
# load a geometry
# search_criteria = {
#     # "productType": "S1_SAR",
#     "productType": "S1_SAR_SLC",
#     "start": "2023-12-20",
#     "end": "2024-01-07",
#     "geom": shp,
# }
# results, _ = dag.search(**search_criteria)

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.

In [ ]:
# TODO: add filter on ascending / descending
explore_products(results, shp)

## Download

In [ ]:
# fill indices of products to download
# indices_to_download = [12, 11]
indices_to_download = [11]
to_download = [results[i] for i in indices_to_download]

# if the data has already been downloaded, just re-run the cell to get the product paths
dl = dag.download_all(to_download, outputs_prefix="/data/S1/", extract=False)

In [ ]:
dag.download_all

# Process

In [ ]:
# France: bigger shape
# shp_proc = shape({"type":"Polygon","coordinates":[[[2.17907,50.791179],[2.368584,50.791179],[2.368584,50.68232],[2.17907,50.68232],[2.17907,50.791179]]]})

In [ ]:
substr = ".SAFE/" 
# substr = ".SAFE/manifest.safe" 
dl0 = dl[0].replace(".zip",substr)
dl1 = dl[1].replace(".zip",substr) 
print(dl1)

In [ ]:
# extract paths for processing
insar_dirs = process_InSAR(
    dl1,
    # dl[0],
    dl0,
    # dl[1],
    outputs_prefix="/data/res",
    tmp_dir="/data/tmp",
    # aoi_name = "flood-Roermond-2021",
    aoi_name = "flood-Roermond-2021-full",
    # aoi_name = "flood-Pas-de-Calais-full-swath",
    # shp=shp,
    pol="full",
    coh_only=True,
    intensity=True,
    clear_tmp_files=True,
    resume=False
)

In [ ]:
insar_dir = "/data/res/S1_InSAR_VH_2021-07-06-172452__2021-07-18-172453_flood-Roermond-2021-full_repro"

In [ ]:
import rasterio
with rasterio.open(f"{insar_dir}/int_mst.tif") as ds_mst:
    mst = ds_mst.read(1)
with rasterio.open(f"{insar_dir}/int_slv.tif") as ds_slv:
    slv = ds_slv.read(1)

In [ ]:
from eo_tools.S1 import change_int

change_int(f"{insar_dir}/int_mst.tif",f"{insar_dir}/int_slv.tif", f"{insar_dir}/change.tif", db=True)

In [ ]:
show_cog(f"{insar_dir}/change.tif", rescale="-1,0", colormap_name="blues_r")
# show_sar_int(f"{insar_dir}")